# Model Performance Analytics

## Imports

In [1]:
from typing import Tuple

import boto3 as aws
import pandas as pd
import seaborn as sns
from botocore.exceptions import ClientError

## Load Data

In [2]:
def download_metrics(aws_bucket: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Get all model and test metrics from AWS S3 bucket.
    
    This function retreives all model and test metrics that have been
    persisted to-date and returns them as two DataFrames.
    """
    def _load_dataset_from_aws_s3(s3_obj_key: str) -> pd.DataFrame:
        """Load CSV datafile from AWS S3 into DataFrame."""
        object_data = s3_client.get_object(
            Bucket=aws_bucket,
            Key=s3_obj_key
        )
        return pd.read_csv(object_data['Body'])

    try:
        s3_client = aws.client('s3')
        print(f'downloading model metrics from s3://{aws_bucket}/model-metrics')
        s3_model_metrics_objects = s3_client.list_objects(
            Bucket=aws_bucket,
            Prefix='model-metrics/'
        )
        model_metrics = pd.concat(
            _load_dataset_from_aws_s3(obj['Key'])
            for obj in s3_model_metrics_objects['Contents']
        )
        print(f'downloading test metrics from s3://{aws_bucket}/model-metrics')
        s3_test_metrics_objects = s3_client.list_objects(
            Bucket=aws_bucket,
            Prefix='test-metrics/'
        )
        test_metrics = pd.concat(
            _load_dataset_from_aws_s3(obj['Key'])
            for obj in s3_test_metrics_objects['Contents']
        )
        test_metrics.reset_index(drop=True, inplace=True)
    except ClientError as e:
        print(f'failed to download training data from s3://{aws_bucket}/datasets')
    return (model_metrics, test_metrics)

Get the metrics!

In [3]:
model_metrics, test_metrics = download_metrics('bodywork-mlops-project')

downloading model metrics from s3://bodywork-mlops-project/model-metrics
downloading test metrics from s3://bodywork-mlops-project/model-metrics


In [4]:
model_metrics

,date,MAPE,r_squared,max_residual
0,2021-04-08,0.780074,0.662754,24.315538


In [5]:
test_metrics

,date,MAPE,r_squared,max_residual,mean_response_time
0,2021-04-08,0.800668,0.80507,126.919421,0.008221
